In [6]:
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
from jax import grad
import jax
import numpy as np
import math
import keras
%matplotlib inline
from jaxmodel import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Test model

In [7]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data(path="mnist.npz", )
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

def batch(x, y, batch_size=32):
    if len(x) % batch_size != 0:
        x = x[:batch_size * (len(x)//batch_size)]
        y=y[:batch_size*(len(x)//batch_size)]
    return np.array_split(x, len(x) / batch_size, axis=0), np.array_split(y, len(y)/batch_size, axis=0)

def fix_data(x, y):
    x = x.reshape(x.shape[0], 28*28)/255
    test = np.zeros((x.shape[0], 10))
    test[np.arange(x.shape[0]),y] = 1
    return (x, test)

fixed_x, fixed_y = fix_data(x_train[:10000], y_train[:10000])
b_x , b_y = batch(fixed_x, fixed_y, 32)



In [ ]:
jaxmodel = JaxModel(28*28, 10, [ 8, 16], mse)
datas = []
for _epoch in range(20):
    print(f"starting epoch {_epoch}")
    datas.append(jaxmodel.train_epoch(b_x, b_y, lr=1e-2 ))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

fig = plt.figure(figsize=(10, 7))
pic = 1
for i, img in enumerate(x_test[:10]):
  plt.subplot(2, 5, pic)
  plt.axis('off')
  predicted = jaxmodel.fd(img.flat)
  plt.title(f"T {y_test[i]} mine {np.argmax(predicted)}")
  plt.imshow(img)
  pic+= 1
plt.show()
